In [ ]:
# IMPORTING PACKAGES
import os
import numpy as np
import pandas as pd
import datetime as dt
#from twilio.rest import Client # to send SMS
import schedule # to schedule the task at a certain time
import time # to set the task to a certain time
from datetime import date
import yfinance as yf
import yfinance.shared as shared
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
#from webserver import keep_alive

In [ ]:
import sys
print(sys.executable)

In [ ]:
# ticker watchlist
ticker_name = {'AAPL':'Apple','MSFT':'Microsoft','GOOG':'Alphabet (Google)','AMZN':'Amazon','FB':'Facebook','TSLA':'Tesla','V':'Visa','NVDA':'Nvidia','JPM':'JP Morgan','JNJ':'Johnson & Johnson','WMT':'Walmart','UNH':'UnitedHealth','MA':'Mastercard','HD':'Home Depot','PG':'Procter & Gamble','PYPL':'PayPal','BAC':'Bank of America','DIS':'Walt Disney','ADBE':'Adobe','NKE':'Nike','CMCSA':'Comcast','LLY':'Eli Lilly','PFE':'Pfizer','ORCL':'Oracle','KO':'Coca-Cola','CRM':'Salesforce','XOM':'Exxon Mobil','CSCO':'Cisco','NFLX':'Netflix','DHR':'Danaher','INTC':'Intel','ABT':'Abbott Laboratories','PEP':'Pepsico','TMO':'Thermo Fisher Scientific','ABBV':'AbbVie','AVGO':'Broadcom','COST':'Costco','CVX':'Chevron','WFC':'Wells Fargo','MS':'Morgan Stanley','TXN':'Texas Instruments','MCD':'McDonalds','MRNA':'Moderna','UPS':'United Parcel Service','QCOM':'QUALCOMM','HON':'Honeywell','NEE':'Nextera Energy','PM':'Philip Morris','BMY':'Bristol-Myers','INTU':'Intuit','UNP':'Union Pacific Corporation','CHTR': 'Charter Communications','C':'Citigroup','SBUX':'Starbucks','AMD':'AMD','BLK':'BlackRock','LOW':"Lowe's Companies",'BA':'Boeing','AXP':'American Express','AMGN':'Amgen','RTX':'Raytheon Technologies','SCHW':'Charles Schwab','AMAT':'Applied Materials','SQ':'Square','GS':'Goldman Sachs','TGT':'Target','AMT':'American Tower','IBM':'IBM','SNAP':'Snapchat','ISRG':'Intuitive Surgical','EL':'Estee Lauder','ZM':'Zoom','NOW':'ServiceNow','MMM':'3M','DE':'Deere & Company','CAT':'Caterpillar','GE':'General Electric','SPGI':'S&P Global','CVS':'CVS Health','LMT':'Lockheed Martin','SYK':'Stryker Corporation','ZTS':'Zoetis','PLD':'Prologis','ABNB':'Airbnb','LRCX':'Lam Research','ANTM':'Anthem','MU':'Micron Technology','BKNG':'Booking.com','ADP':'Automatic Data Processing','MO':'Mondelez','CCI':'Crown Castle','BX':'Blackstone Group','DUK':'Duke Energy'}

tickers = list(ticker_name.keys()) # tickers

moving_averages = ['SMA50','SMA100','SMA200']
stocks_bought = {} # list storing all stocks that have had a buy signal but no sell signal yet
stocks_traded = [] # list storing all stocks ever traded, including: buy price and date, and the same for sell # was [] brackets before
print(stocks_bought)
print(stocks_traded)

In [ ]:
# Keep this repl running forever using uptime robot
#keep_alive()

def get_data():
    
    # Downloading the data
    print('data gathered')
    downloaded_data = yf.download(tickers,interval='1d',period='5y') # download the data
    
    # Remove tickers with no data
    failed_tickers = shared._ERRORS.keys() # see which tickers have no data whatsoever
    for failed_ticker in failed_tickers:
        downloaded_data.drop([('Adj Close',failed_ticker),('Close',failed_ticker),('High',failed_ticker),('Low',failed_ticker),('Open',failed_ticker),('Volume',failed_ticker)], axis=1) # remove the failed tickers from the dataframe

    # Remove tickers that have more than 8% nan values in the last 200 days and/or have no position in currently
    for ticker_not_bought in list(set(tickers) - set(list(stocks_bought.keys()))):
      if (downloaded_data.iloc[-200:].loc[:,('Adj Close',ticker_not_bought)].isna().sum() / 200) > 0.08:
        downloaded_data.drop([('Adj Close',ticker_not_bought),('Close',ticker_not_bought),('High',ticker_not_bought),('Low',ticker_not_bought),('Open',ticker_not_bought),('Volume',ticker_not_bought)], axis=1) # remove the failed tickers from the dataframe

    # Removing nan values by forwardfill and backfill
    downloaded_data.fillna(method='ffill', axis=0, inplace=True)
    downloaded_data.fillna(method='bfill', axis=0, inplace=True)

    # Creating an empty dataframe to store the correct data
    processed_data = pd.DataFrame(index=downloaded_data.index, columns=pd.MultiIndex.from_product([tickers,['Adj Close']+moving_averages]))
    
    for stock in tickers:
        
      # Inserting close data
      processed_data.loc[:,(stock,'Adj Close')] = downloaded_data.loc[:,('Adj Close',stock)]
      
      # Working out and inserting moving averages values
      for sma in moving_averages:
        # sma[3:] is the period (eg. 50, 100, or 200)
        processed_data.loc[:,(stock,sma)] = downloaded_data.loc[:,('Adj Close',stock)].rolling(int(sma[3:])).mean()
    
    # Drop the most of the history and leave only the last 4 weeks of data.
    processed_data = processed_data.iloc[-4:]

    # Put most recent data at the top of the list
    processed_data = processed_data.iloc[::-1]

    # Reset the index and drop the old one
    processed_data = processed_data.reset_index()
    processed_data = processed_data.drop(('Date',''), axis=1)

    # Check if today is the last row in processed data. If yes then check for signals
    check_signals(processed_data)


In [ ]:
#Set the email address to send and receive mail from
senderh_email = "xxxxxx@outlook.com" #The sender email, change hotmail to outlook to use the backup email. @hotmail.com is the original
senderg_email = "xxxxx@gmail.com"
rec_email = "xxxxx@yahoo.com" #The receiver email
senderh_email_password = "abcdefgh" #The password to the sender email
senderg_email_password = "abcdefgh"

In [ ]:
def send_mail(subject, message):

    username = senderh_email
    password = senderh_email_password
    recipient = rec_email
    

    msg = MIMEMultipart()
    msg['From'] = username
    msg['To'] = recipient
    msg['Subject'] = subject
    msg.attach(MIMEText(message))

    try:
        print('sending mail to ' + recipient + ' on ' + subject)

        mailServer = smtplib.SMTP('smtp-mail.outlook.com', 587)
        #mailServer = smtplib.SMTP('smtp.gmail.com', 587)
        mailServer.ehlo()
        mailServer.starttls()
        mailServer.ehlo()
        mailServer.login(username, password)
        mailServer.sendmail(username, recipient, msg.as_string())
        mailServer.close()

    except Exception as e:
        print(str(e))


In [ ]:
def check_signals(processed_input: pd.DataFrame()):
  print('called check signals')
  # CHECK ALL STOCKS FOR SIGNALS
  for stock in tickers:

    close = processed_input.loc[:,(stock,'Adj Close')]  
    sma200 = processed_input.loc[:,(stock,'SMA200')]
    sma100 = processed_input.loc[:,(stock,'SMA100')]
    sma50 = processed_input.loc[:,(stock,'SMA50')]

    lowest_ma = sma50 # Make SMA 50 the lowest moving average for the buy signal 
    highest_ma = sma50 # Make SMA 50 the lowest moving average for the sell signal
    lowest_ma_string = ''
    highest_ma_string = ''
      
    # Finding the lowest simple moving average (SMA)
    if sma50[2] < sma100[2]:
        # SMA50 < SMA100
        if sma50[2] < sma200[2]:
            # SMA50 <SMA100 && SMA200
            lowest_ma = sma50
            lowest_ma_string = 'SMA50'
            highest_ma = sma200
            highest_ma_string = 'SMA200'
        else:
            # SMA200 > SMA50 < SMA100
            lowest_ma = sma200
            lowest_ma_string = 'SMA200'
            highest_ma = sma50
            highest_ma_string = 'SMA50' 
    else:
        # SMA100 < SMA50
        if sma100[2] < sma200[2]:
            # SMA100 is less than SMA50 and SMA200
            lowest_ma = sma100
            lowest_ma_string = 'SMA100'
        else:
            # SMA100 is less than SMA50 but not less than SMA200. So SMA200 is the lowest
            lowest_ma = sma200
            lowest_ma_string = 'SMA200'
    
    # Finding the crossover between the price and the lowest moving average
    if close[0] > lowest_ma[0]:

      if close[1] > lowest_ma[1]:

        if close[2] > lowest_ma[2]:

          if close[3] < lowest_ma[3]:
            # It has been three days since the stock crossed over. Time to buy!
            stocks_bought[stock] = [date.today(), close[0]]
            stocks_traded.append([stock, 'buy', date.today(), close[0]])
            
            TIME_TO_BUY_SUBJECT = "Time to buy!"
            TIMETOBUY_message = "{} ({}) crossed above the {} four days ago and has been above since. {} closed at a price of ${} today. Remember to buy tomorrow!".format(ticker_name[stock], stock, lowest_ma_string, stock, close[0])
            send_mail(TIME_TO_BUY_SUBJECT, TIMETOBUY_message)
            print("Email was sent") #confirms that an email was sent
            continue # end this iteration of the loop without checking for a rebound off the lowest SMA
        
        else:
          # It has been above the MA for two days
          ABOVEMA_SUBJECT = "Keep a look out! Crossed above yesterday - above MA for 2 days and is appreciating in value!"
          aboveMA_message = "{} ({}) has stayed above the {} for two days in a row. {} closed at a price of ${} today. I will let you know if it closes above the {} tomorrow.".format(ticker_name[stock], stock, lowest_ma_string, stock, close[0], lowest_ma_string)
          send_mail(ABOVEMA_SUBJECT, aboveMA_message)
          print("Email was sent") #confirms that the email was sent   
          continue # end this iteration of the loop without checking for a rebound off the lowest SMA
      else:
        # Crossover today
        CROSSOVER_ABOVE_SUBJECT = "Stock closed above lowest MA today, but not yesterday - CROSSOVERABOVE!"
        Crossoverabove_message = "{} ({}) crossed above the {} today! {} closed at a price of ${}.".format(ticker_name[stock],stock,lowest_ma_string,stock,close[0])
        send_mail(CROSSOVER_ABOVE_SUBJECT, Crossoverabove_message)
        print("Email was sent") #confirms that the email was sent 
        continue # end this iteration of the loop without checking for a rebound off the lowest SMA

  # CHECK BOUGHT STOCKS FOR SELL SIGNALS
  for stock in list(stocks_bought):
    # Sell if the closing price went below the SMA50 or SMA200 three days ago
    # Check if close < today's ma
    if close[0] < highest_ma[0]:
      # Check if close < yesterday's ma
      if close[1] < highest_ma[1]:
        # Check if close < three days ago ma
        if close[2] < highest_ma[2]:
          # Check if close > three days ago to detect crossover
          if close[3] > highest_ma[3]:
            ### Time to sell!
            delta = date.today() - stocks_bought[stock][0] # number of days the trade was held
            profit = round((close[0]/stocks_bought[stock][1])-1,2)*100 # calculate profit or loss

            stocks_bought.pop(stock)
            stocks_traded.append([stock, 'sell', date.today(), close[0]])
            
            if profit > 0:
              # Profit
              PROFIT_MESSAGE = "{} ({}) crossed below the {} four days ago and has been below since. {} closed at a price of ${} today. That's a {}% profit in "+str(delta.days)+" days! Congrats! Remember to sell tomorrow!".format(ticker_name[stock], stock, lowest_ma_string, stock, close[0], profit) #send the email with your custom message
              PROFIT_SUBJECT = "You made a profit, SELL THE STOCKS!"
              send_mail(PROFIT_SUBJECT, PROFIT_MESSAGE)
              print("Email was sent") #confirms that the email was sent  
            else:
              # Loss
              LOSS_MESSAGE = "{} ({}) crossed below the {} four days ago and has been below since. {} closed at a price of ${} today. Unfortunately, that's a {}% loss. Remember to sell tomorrow.".format(ticker_name[stock], stock, lowest_ma_string, stock, close[0], profit)
              LOSS_SUBJECT = "You made a loss, SELL THE STOCKS"
              print("Email was sent") #confirms that the email was sent 
        else:
          # Below the MA for two days
          MA_BELOW_SUBJECT = "Keep a look out! Crossed below yesterday - below MA for 2 days and is depreciating in value!"
          MA_BELOW_MESSAGE = "{} ({}) has stayed below the {} for two days in a row. {} closed at a price of ${} today.".format(ticker_name[stock], stock, lowest_ma_string, stock, close[0])
          send_mail(MA_BELOW_SUBJECT, MA_BELOW_MESSAGE)
          print("Email was sent") #confirms that the email was sent 
      else:
        # Crossover today
        CROSSOVER_BELOW_SUBJECT = "Stock closed below lowest MA today, but not yesterday - CROSSOVERBELOW!"
        CROSSOVER_BELOW_MESSAGE = "{} ({}) crossed below the {} today! {} closed at a price of ${}.".format(ticker_name[stock], stock, lowest_ma_string, stock, close[0])
        send_mail(MA_BELOW_SUBJECT, MA_BELOW_MESSAGE)
        print("Email was sent") #confirms that the email was sent 

In [ ]:
TEST_SUBJECT = "It Worked"
TEST_MSG = "It Worked!"
send_mail(TEST_SUBJECT, TEST_MSG)

In [ ]:
%%time
# SCHEDULE WHEN TO RUN EVERYTHING
## The schedule does not clear even if you cancel a run and start it again so need to clear the schedule every time it is run and then re-declare the schedule
schedule.clear()

schedule.every().hour.do(get_data)

# Keep the schedule running perpetually. The while loop checks if there is task the scheduler must run.
while True:
    schedule.run_pending()
    time.sleep(1)